<a href="https://colab.research.google.com/github/Neil-Cardoz/Deep-Learning/blob/main/Ex_4_Iris_w_o_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Iris without Tensonflow

In [13]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelBinarizer

In [14]:
# Load and Preprocess the Iris dataset
data = pd.read_csv('/content/Iris.csv')
label_binarizer = LabelBinarizer()
y = label_binarizer.fit_transform(data['Species'])
X = data.drop(columns=['Id', 'Species'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [15]:
# Initialize network
def initialize_network(input_size, hidden_size, output_size):
    """Initialize a neural network with given sizes for input, hidden, and output layers."""
    np.random.seed(42)
    network = []
    layer_sizes = [input_size] + hidden_size + [output_size]
    for i in range(len(layer_sizes) - 1):
        layer = {
            'weights': np.random.randn(layer_sizes[i], layer_sizes[i + 1]) * 0.1,
            'biases': np.zeros((1, layer_sizes[i + 1]))
        }
        network.append(layer)
    return network

input_size = X_train.shape[1]
hidden_size = [8, 8]  # Two hidden layers with 8 neurons each
output_size = y_train.shape[1]
network = initialize_network(input_size, hidden_size, output_size)

In [16]:
# Define Activation Functions
def sigmoid(x):
    """Sigmoid activation function."""
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    """Derivative of the sigmoid function."""
    return x * (1 - x)

# Forward Propagation
def forward_propagation(network, inputs):
    """Perform forward propagation through the network."""
    activation = inputs
    for layer in network:
        z = np.dot(activation, layer['weights']) + layer['biases']
        activation = sigmoid(z)
        layer['activation'] = activation
    return activation

# Backward Propagation
def backward_propagation(network, inputs, targets, learning_rate):
    """Perform backward propagation and update weights and biases."""
    forward_propagation(network, inputs)
    for i in reversed(range(len(network))):
        layer = network[i]
        if i == len(network) - 1:
            # Output layer
            layer['delta'] = (layer['activation'] - targets) * sigmoid_derivative(layer['activation'])
        else:
            next_layer = network[i + 1]
            layer['delta'] = np.dot(next_layer['delta'], next_layer['weights'].T) * sigmoid_derivative(layer['activation'])

    for i in range(len(network)):
        layer = network[i]
        if i == 0:
            inputs_to_use = inputs
        else:
            inputs_to_use = network[i - 1]['activation']
        layer['weights'] -= learning_rate * np.dot(inputs_to_use.T, layer['delta'])
        layer['biases'] -= learning_rate * np.sum(layer['delta'], axis=0, keepdims=True)

In [17]:
# Train the network
def train_network(network, inputs, targets, learning_rate, epochs):
    """Train the neural network using the provided inputs and targets."""
    for epoch in range(epochs):
        backward_propagation(network, inputs, targets, learning_rate)
        if (epoch + 1) % 100 == 0:
            loss = np.mean(np.square(targets - forward_propagation(network, inputs)))
            print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss:.4f}')

train_network(network, X_train, y_train, learning_rate=0.01, epochs=1000)

Epoch 100/1000, Loss: 0.2221
Epoch 200/1000, Loss: 0.2218
Epoch 300/1000, Loss: 0.2193
Epoch 400/1000, Loss: 0.1465
Epoch 500/1000, Loss: 0.1103
Epoch 600/1000, Loss: 0.1006
Epoch 700/1000, Loss: 0.0860
Epoch 800/1000, Loss: 0.0557
Epoch 900/1000, Loss: 0.0319
Epoch 1000/1000, Loss: 0.0222


In [18]:
# Evaluate the network
def predict(network, inputs):
    """Make predictions using the trained network."""
    outputs = forward_propagation(network, inputs)
    return outputs

predictions = predict(network, X_test)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(y_test, axis=1)
accuracy = np.mean(predicted_classes == true_classes)
print(f'Accuracy: {accuracy * 100:.2f}%')
print(f'Predictions: {predictions}')
print(f'Predicted Classes: {predicted_classes}')

Accuracy: 100.00%
Predictions: [[4.49070366e-02 8.22723797e-01 7.79783514e-02]
 [9.49755651e-01 1.09407356e-01 2.26802833e-04]
 [1.79864403e-03 8.41275167e-02 9.63773509e-01]
 [2.60789346e-02 7.29036260e-01 1.78066107e-01]
 [2.58939818e-02 7.41236316e-01 1.75766840e-01]
 [9.45464797e-01 1.16449562e-01 2.44928989e-04]
 [1.12655090e-01 8.32611169e-01 2.28829429e-02]
 [2.76369368e-03 1.33784971e-01 9.23537892e-01]
 [1.09821778e-02 5.02204349e-01 4.90749820e-01]
 [7.57347332e-02 8.54113988e-01 3.62939471e-02]
 [4.72510321e-03 2.34238210e-01 8.20678878e-01]
 [9.45278613e-01 1.17046566e-01 2.44810121e-04]
 [9.49598989e-01 1.09869651e-01 2.26919974e-04]
 [9.46170077e-01 1.15552209e-01 2.41176331e-04]
 [9.50757520e-01 1.07776342e-01 2.22454753e-04]
 [3.09241532e-02 7.38776805e-01 1.46872548e-01]
 [2.27750028e-03 1.08318553e-01 9.45023090e-01]
 [7.01333284e-02 8.60397598e-01 3.90652790e-02]
 [3.94606593e-02 8.01957472e-01 9.60532890e-02]
 [2.24560163e-03 1.06892891e-01 9.46264387e-01]
 [9.46053